In [1]:
import ml_framework
from ml_framework import dataset, preprocess, feature, model

In [2]:
import seaborn.apionly as sns
import pandas as pd

@dataset(train_valid_test=(0.6, 0.2, 0.2))
def raw_dataset():
    titanic_dataset = sns.load_dataset('titanic')

    # Drop NaN rows for simplicity
    titanic_dataset.dropna(inplace=True)

    # Extract X and y
    X = titanic_dataset.drop('survived', axis=1)
    y = titanic_dataset['survived']
    return X, y

# my_dataset is now a variable that holds the X values of the evaluated function
# (the test data's ground truth is locked away to prevent accidentially fitting to it)
raw_dataset.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
3,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
6,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
10,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True


In [3]:
@preprocess
def preprocessed_dataset(X):
    # Encode categorical columns
    categorical_column_names = [
        'sex', 'embarked', 'class',
        'who', 'adult_male', 'deck',
        'embark_town', 'alive', 'alone'
    ]

    X = pd.get_dummies(X,
                       columns=categorical_column_names,
                       prefix=categorical_column_names)

    return X

preprocessed_dataset.head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,...,deck_E,deck_F,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,alive_no,alive_yes,alone_False,alone_True
1,1,38.0,1,0,71.2833,1,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0
3,1,35.0,1,0,53.1000,1,0,0,0,1,...,0,0,0,0,0,1,0,1,1,0
6,1,54.0,0,0,51.8625,0,1,0,0,1,...,1,0,0,0,0,1,1,0,0,1
10,3,4.0,1,1,16.7000,1,0,0,0,1,...,0,0,1,0,0,1,0,1,1,0
11,1,58.0,0,0,26.5500,1,0,0,0,1,...,0,0,0,0,0,1,0,1,0,1


In [4]:
from sklearn import decomposition
import numpy as np

@feature('pca')
def pca_feature(X):
    pca = decomposition.PCA(n_components=3)
    pca.fit(X)
    pca_out = pca.transform(X)
    
    pca_out = np.transpose(pca_out, (1, 0))
    return {'pca_0': pca_out[0], 'pca_1': pca_out[1], 'pca_2': pca_out[2]}

pca_feature.head()

,pca_0,pca_1,pca_2
1,-7.671430,2.211611,-1.524306
3,-25.798643,-1.153763,-1.330469
6,-27.419192,17.883788,1.170164
10,-61.594390,-32.894045,-0.423018
11,-52.795269,21.329436,-1.718752


In [5]:
import xgboost as xgb

@model('xgboost')
def xgboost_model():
    def define(num_columns):
        return None # xgboost models are not pre-defined
    
    def train(model, params, train, validation):
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'logloss'
        
        d_train = xgb.DMatrix(train['X'], label=train['y'])
        d_valid = xgb.DMatrix(validation['X'], label=validation['y'])

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]

        return xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=50, verbose_eval=200)
    
    def predict(model, X):
        return model.predict(xgb.DMatrix(X))
    
    return define, train, predict

In [6]:
ml_framework.train('xgboost', {
    'max_depth': 7,
    'eta': 0.005
})

[0]	train-logloss:0.6885	valid-logloss:0.688485
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[200]	train-logloss:0.221082	valid-logloss:0.220172
[400]	train-logloss:0.087941	valid-logloss:0.087103
[600]	train-logloss:0.040761	valid-logloss:0.040085
[800]	train-logloss:0.022548	valid-logloss:0.022152
[1000]	train-logloss:0.019019	valid-logloss:0.019206
[1200]	train-logloss:0.018926	valid-logloss:0.019022
[1400]	train-logloss:0.018901	valid-logloss:0.018951
[1600]	train-logloss:0.018894	valid-logloss:0.018921
[1800]	train-logloss:0.018892	valid-logloss:0.018907
[2000]	train-logloss:0.018892	valid-logloss:0.0189
Stopping. Best iteration:
[2075]	train-logloss:0.018892	valid-logloss:0.018898



In [7]:
ml_framework.evaluate('xgboost')

0.02013238549635217

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

@model('keras_nn')
def keras_nn_model():
    def define(num_columns):
        model = Sequential()
        model.add(Dense(64, input_dim=num_columns, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])
        return model

    
    def train(model, params, train, validation):
        model.fit(train['X'].values, train['y'].values,
                  epochs=50,
                  batch_size=5)
        
        return model
    
    def predict(model, X):
        return model.predict(X.values)
    
    return define, train, predict

In [17]:
ml_framework.train('keras_nn', {})

Epoch 1/50
108/108 [==============================] - 0s - loss: 2.4998 - acc: 0.6296     
Epoch 2/50
108/108 [==============================] - 0s - loss: 2.1377 - acc: 0.5741     
Epoch 3/50
108/108 [==============================] - 0s - loss: 1.5453 - acc: 0.6019     
Epoch 4/50
108/108 [==============================] - 0s - loss: 1.8973 - acc: 0.6111     
Epoch 5/50
108/108 [==============================] - 0s - loss: 1.4034 - acc: 0.6481     
Epoch 6/50
108/108 [==============================] - 0s - loss: 1.2242 - acc: 0.6481     
Epoch 7/50
108/108 [==============================] - 0s - loss: 1.2834 - acc: 0.6111     
Epoch 8/50
108/108 [==============================] - 0s - loss: 1.3846 - acc: 0.5556     
Epoch 9/50
108/108 [==============================] - 0s - loss: 1.0911 - acc: 0.5833     
Epoch 10/50
108/108 [==============================] - 0s - loss: 1.0862 - acc: 0.6204     
Epoch 11/50
108/108 [==============================] - 0s - loss: 1.1080 - acc: 0.6759   

In [18]:
ml_framework.evaluate('keras_nn')

0.56353224844143202